In [3]:
import tensorflow as tf

# Définir les transformations contrastives
class ContrastiveTransformations(tf.keras.layers.Layer):
    def __init__(self, base_transforms, n_views=2):
        super(ContrastiveTransformations, self).__init__()
        self.base_transforms = base_transforms
        self.n_views = n_views

    def call(self, x):
        # Appliquer les transformations pour générer plusieurs vues
        return [self.base_transforms(x) for i in range(self.n_views)]

# Définir les transformations de base
contrast_transforms = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomContrast(0.2),
    tf.keras.layers.Rescaling(1./255)
])




from tensorflow.keras import layers, models

# Définir le modèle CNN
def create_cnn_model(input_shape, num_classes):
    model = models.Sequential()

    # Bloc 1
    model.add(layers.Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
    model.add(layers.Activation('relu'))
    model.add(layers.Conv2D(32, (3, 3)))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.3))

    # Bloc 2
    model.add(layers.Conv2D(64, (3, 3), padding='same'))
    model.add(layers.Activation('relu'))
    model.add(layers.Conv2D(64, (3, 3)))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.3))

    # Bloc 3
    model.add(layers.Flatten())
    model.add(layers.Dense(512))
    model.add(layers.Activation('relu'))
    model.add(layers.Dropout(0.5))

    return model

# Modèle CNN sans la couche de classification, juste l'extraction de caractéristiques
cnn_model = create_cnn_model(input_shape=(96, 96, 3), num_classes=10)

# Ajouter une tête de projection MLP
class ProjectionHead(tf.keras.layers.Layer):
    def __init__(self, projection_dim=128, **kwargs):
        super(ProjectionHead, self).__init__(**kwargs)
        self.projection_dim = projection_dim
        self.dense1 = layers.Dense(512, activation='relu')
        self.dense2 = layers.Dense(projection_dim, activation=None)  # Projection en un espace plus petit

    def call(self, inputs):
        x = self.dense1(inputs)
        return self.dense2(x)

# Ajouter la tête de projection au modèle CNN
projection_head = ProjectionHead(projection_dim=128)


C:\Users\Ninette HOUKPONOU\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
import tensorflow as tf

# Fonction de perte Contrastive Loss (NT-Xent)
def contrastive_loss(projections1, projections2, temperature=0.5):
    # Calculer la similarité entre les projections à l'aide du produit scalaire
    dot_product = tf.matmul(projections1, projections2, transpose_b=True)
    
    # Appliquer le facteur de température
    logits = dot_product / temperature
    
    # Calculer la perte NT-Xent
    labels = tf.eye(logits.shape[0], dtype=tf.float32)  # Matrice d'identité pour la vérité terrain (positives en diagonale)
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)
    
    return tf.reduce_mean(loss)


In [ ]:
import tensorflow as tf

# Définir un optimiseur
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

# Préparer les données d'entraînement (exemple avec des images fictives)
# 
train_images = ...  # Charger tes images d'entraînement
train_labels = ...  # Charger tes labels (si nécessaire)

# Appliquer les transformations contrastives sur les images
contrastive_layer = ContrastiveTransformations(contrast_transforms, n_views=2)

# Boucle d'entraînement
for epoch in range(num_epochs):
    for step, image_batch in enumerate(train_images):
        # Générer des vues augmentées
        augmented_images = contrastive_layer(image_batch)
        
        with tf.GradientTape() as tape:
            # Passer les images augmentées dans le modèle pour obtenir les projections
            projections1 = cnn_model(augmented_images[0], training=True)
            projections2 = cnn_model(augmented_images[1], training=True)

            # Passer les projections par la tête de projection
            projections1 = projection_head(projections1)
            projections2 = projection_head(projections2)

            # Calculer la perte contrastive
            loss = contrastive_loss(projections1, projections2)
        
        # Calculer et appliquer les gradients
        gradients = tape.gradient(loss, cnn_model.trainable_variables + projection_head.trainable_variables)
        optimizer.apply_gradients(zip(gradients, cnn_model.trainable_variables + projection_head.trainable_variables))

        if step % 100 == 0:
            print(f"Epoch {epoch}, Step {step}, Loss: {loss.numpy()}")

